In [1]:
from agno.models.openai import OpenAILike
from agno.agent import Agent
import uuid
import requests
from dotenv import load_dotenv
import os
from agno.os import AgentOS

# Load environment variables from .env file
load_dotenv()

True

In [2]:
from pydantic_settings import BaseSettings
from pydantic import Field


class Settings(BaseSettings):
    """Application settings loaded from environment variables"""

    client_app_secret: str = Field(
        ..., description="Client application secret for authentication"
    )
    experience_id: str = Field(..., description="Intuit experience ID")
    client_app_id: str = Field(..., description="Client application ID")
    profile_id: str = Field(..., description="Profile ID for authentication")

    # Pydantic v2 uses model_config instead of inner Config class
    model_config = {
        "env_file": ".env",
        "env_file_encoding": "utf-8",
        "case_sensitive": False,
        "extra": "ignore",  # Ignore extra env vars like SERPER_API_KEY
    }


# Initialize settings
settings = Settings()

In [3]:
def _get_auth_token():
    """Get cached authentication token or fetch a new one if not available"""

    # Fetch new token from IAM
    url = "https://identityinternal.api.intuit.com/v1/graphql"

    IAM_MUTATION = """mutation identitySignInInternalApplicationWithPrivateAuth($input: Identity_SignInApplicationWithPrivateAuthInput!) {
        identitySignInInternalApplicationWithPrivateAuth(input: $input) {
            authorizationHeader
        }
    }"""

    headers = {
        "intuit_tid": str(uuid.uuid4()),
        "Authorization": f"Intuit_IAM_Authentication intuit_appid={settings.client_app_id}, intuit_app_secret={settings.client_app_secret}",
        "Content-Type": "application/json",
    }

    data = {
        "query": IAM_MUTATION,
        "variables": {"input": {"profileId": settings.profile_id}},
    }

    try:
        response = requests.post(url, json=data, headers=headers)
        response.raise_for_status()
        result = response.json()

        base_token = result["data"]["identitySignInInternalApplicationWithPrivateAuth"][
            "authorizationHeader"
        ]
        _cached_token = f"{base_token},intuit_appid={settings.client_app_id},intuit_app_secret={settings.client_app_secret}"
        print("✓ New auth token obtained and cached")
        return _cached_token
    except Exception as e:
        print(f"✗ Failed to get auth token: {e}")
        raise

In [4]:
def get_llm(model_name="amazon.nova-pro-v1-0"):
    llm = OpenAILike(
        base_url=f"https://llmexecution.api.intuit.com/v3/lt/{model_name}",
        extra_headers={
            "intuit_experience_id": settings.experience_id,
            "intuit_originating_assetalias": "Intuit.coe.pecomplianceremediation",
            "Authorization": _get_auth_token(),
        },
    )
    return llm

In [5]:
from agno.vectordb.chroma import ChromaDb
from agno.knowledge import Knowledge, embedder
from agno.knowledge.embedder.sentence_transformer import SentenceTransformerEmbedder
from agno.knowledge.chunking.fixed import FixedSizeChunking
from agno.knowledge.reader.pdf_reader import PDFReader

kn = Knowledge(
    name="Financial Documents",
    description="This is a knowledge base for financial documents which includes income tax documents, personal financing documents, etc.",
    vector_db=ChromaDb(
        collection="financial_documents",
        path="tmp/chroma",
        persistent_client=True,
        embedder=SentenceTransformerEmbedder(id="all-MiniLM-L6-v2"),
    ),
)

reader = PDFReader(
    chunk_size=500, chunking_strategy=FixedSizeChunking(chunk_size=500, overlap=100)
)

/Users/ssahu11/conda/miniconda3/envs/bits/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|██████████| 103/103 [00:00<00:00, 2527.37it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-v2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


In [6]:
kn.add_content(
    path="/Users/ssahu11/Personal/BITS/Sem4/Dessertation/FinAgent/data/tax_regimes_comparison.pdf",
    reader=reader,
    metadata={"type": "income_tax_documents"},
)

INFO Adding content from path, f23124c7-1873-5b00-90c4-3a2abd827f23, None,                                         
     /Users/ssahu11/Personal/BITS/Sem4/Dessertation/FinAgent/data/tax_regimes_comparison.pdf, None

INFO Upserting 95 documents

WARNING  Contents DB not found for knowledge base: Financial Documents

In [7]:
kn.add_content(
    path="/Users/ssahu11/Personal/BITS/Sem4/Dessertation/FinAgent/data/personal_finance_manual_2026.pdf",
    reader=reader,
    metadata={"type": "personal_financing_documents"},
)

INFO Adding content from path, ee132c56-d046-598c-addc-0583350d4eff, None,                                         
     /Users/ssahu11/Personal/BITS/Sem4/Dessertation/FinAgent/data/personal_finance_manual_2026.pdf, None

INFO Upserting 80 documents

WARNING  Contents DB not found for knowledge base: Financial Documents

### Test

In [8]:
from agno.agent import Agent

kn_agent = Agent(
    role="Knowledge Base Agent",
    instructions="You are a knowledge base agent that can help with a wide range of tasks.",
    model=get_llm(),
    knowledge=kn,
    search_knowledge=True,
)

kn_agent.run("tell me about 80c").content

✓ New auth token obtained and cached


INFO Found 10 documents

'\nSection 80C of the Indian Income Tax Act allows taxpayers to claim deductions on certain investments and expenditures, up to a maximum of ₹1.5 lakh per financial year. Here are some common investments and expenditures that qualify under Section 80C:\n\n1. **Employee Provident Fund (EPF)**: Mandatory salary deduction (12% of Basic).\n2. **Public Provident Fund (PPF)**: A long-term investment with a lock-in period of 15 years.\n3. **National Savings Certificate (NSC)**: A government-backed investment with a lock-in period of 5 years.\n4. **Equity Linked Savings Scheme (ELSS)**: Mutual funds with a lock-in period of 3 years.\n5. **Life Insurance Premiums**: Premiums paid for life insurance policies.\n6. **Tuition Fees**: Fees paid for the education of children.\n7. **Senior Citizen Savings Scheme (SCSS)**: A government-backed investment for senior citizens with a lock-in period of 5 years.\n8. **Sukanya Samriddhi Yojana**: A government-backed investment for the education and marriage o